# Context-FID Score Presentation
## Necessary packages and functions call
# 固定ts2vec权重

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [8]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import calculate_fid
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss
from Models.ts2vec.ts2vec import TS2Vec

## Data Loading

Load original dataset and preprocess the loaded data.

In [22]:
iterations = 5
ori_data = np.load('./OUTPUT/energyBL/samples/energy_norm_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('./OUTPUT/Mamenergy/ddpm_fake_Mamenergy.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [23]:
def Context_FID(ori_data, generated_data, model):
    ori_represenation = model.encode(ori_data, encoding_window='full_series')
    gen_represenation = model.encode(generated_data, encoding_window='full_series')
    idx = np.random.permutation(ori_data.shape[0])
    ori_represenation = ori_represenation[idx]
    gen_represenation = gen_represenation[idx]
    results = calculate_fid(ori_represenation, gen_represenation)
    return results

model = TS2Vec(input_dims=ori_data.shape[-1], device='cuda:0', batch_size=8, lr=0.001, output_dims=320, max_train_length=3000)
#model.fit(ori_data, verbose=False)
#model.save('ts2vec_custom.pth')
model.load('ts2vec_energy.pth')
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]],model)
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.07553011601893575 

Iter 1:  context-fid = 0.07553003033652236 

Iter 2:  context-fid = 0.07552981054500527 

Iter 3:  context-fid = 0.07552976211635568 

Iter 4:  context-fid = 0.07552962800478444 

Final Score:  0.07552986940432069 ± 2.483070372286939e-07


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [4]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [5]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.019852216581699826 

Iter 1:  cross-correlation = 0.01816951370252664 

Iter 2:  cross-correlation = 0.022373661672091448 

Iter 3:  cross-correlation = 0.012407943886992933 

Iter 4:  cross-correlation = 0.010309792931556355 

Final Score:  0.01662262575497344 ± 0.006316425881906014
